In [ ]:
!pip install -U pillow

In [ ]:
!pip install rembg

In [ ]:
import requests
from PIL import Image
from io import BytesIO
import os
from rembg.bg import remove

In [ ]:
def process_image(input_path, output_folder, background_path):
    # Load the image
    img = Image.open(input_path)
    
    # Save the original image
    original_filename = os.path.basename(input_path)
    img.save(os.path.join(output_folder, 'original', original_filename), format='JPEG')
    
    # Remove the background
    with open(input_path, 'rb') as f:
        input_data = f.read()
        subject = remove(input_data, alpha_matting=True)
    
    # Save the masked image
    output_path = os.path.join(output_folder, 'masked', original_filename)
    with open(output_path, 'wb') as f:
        f.write(subject)
    
    # Open background image
    background_img = Image.open(background_path)
    background_img = background_img.resize((img.width, img.height))
    
    # Open masked image
    foreground_img = Image.open(output_path)
    
    # Paste masked image onto background
    background_img.paste(foreground_img, (0, 0), foreground_img)
    
    # Save final image
    background_img.save(os.path.join(output_folder, 'processed', original_filename), format='JPEG')


In [ ]:
# Function to process all images in a folder
def process_folder(input_folder, output_folder, background_path):
    os.makedirs(os.path.join(output_folder, 'original'), exist_ok=True)
    os.makedirs(os.path.join(output_folder, 'masked'), exist_ok=True)
    os.makedirs(os.path.join(output_folder, 'processed'), exist_ok=True)
    
    # Iterate over each file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".jpeg") or filename.endswith(".jpg"):
            input_path = os.path.join(input_folder, filename)
            process_image(input_path, output_folder, background_path)


In [ ]:
# Example usage
input_folder = r"D:\My projects\Rembg\frames_extraction_HD\output_frames"
output_folder = r"D:\My projects\Rembg\frames_extraction_HD\processed_frames"
background_path = r'D:\My projects\Rembg\frames_extraction_HD\sunset.jpg'


process_folder(input_folder, output_folder, background_path)